# RF Chain and Channelisation Commissioning Test

## Table of Contents\n
#### [0.1 - Global Configuration](#02---setup-test-equipment)
#### [0.2 - Setup Test Equipment](#02---setup-test-equipment)
#### [1 -> Test Steps](#1---test-steps)
this notebook provides for indepenent control of the test equipment during the rf chain and channelisation test so that test equipment setup may be changed while the telescope is scanning.  All test step requirements are indicated but only test equipment steps are included in this notebook.

## 0.1 Global Configuration

### 0.1.1 Import dependencies

In [2]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
from typing import List

import ska_ser_logging
from bokeh.io import output_notebook
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec, get_default_target_specs_sb
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment

### 0.1.2 Setup Global Variables and Configuration

In [4]:
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}")
timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir, exist_ok=True)
# we disable qa as it is not been properly verified
disable_qa()

Test Equipment Configured: TangoTestEquipment{namespace=test-equipment; tango_host=tango-databaseds.test-equipment.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.test-equipment.svc.miditf.internal.skao.int:8765}


### 0.1.3 Test Connections to Namespaces

In [5]:
test_equipment.smoke_test()

CIA PingResponse (test-equipment): {"result":"ok","time":"2024-07-25T10:55:43.962591"}
mid-itf/siggen/1 is reachable
mid-itf/progattenuator/1 is reachable
mid-itf/spectana/1 is reachable
mid-itf/skysimctl/4 is reachable


## 0.2 Test Equipment Setup

Use the noise source at nominal levels for input to at least one SPFRx.  

### 0.2.1 Configure Test Equipment State 

In [6]:
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

['mid-itf/progattenuator/1',
 'mid-itf/siggen/1',
 'mid-itf/skysimctl/4',
 'mid-itf/spectana/1',
 'taranta/benchmark/1',
 'taranta/test/1',
 'taranta/test/2',
 'taranta/test/3',
 'test/device/1']

### 0.2.2 Print Test Equipment Diagnostics

In [7]:
test_equipment.print_diagnostics()

mid-itf/skysimctl/4 State: ON
mid-itf/skysimctl/4 Band: 1
mid-itf/skysimctl/4 Correlated_Noise_Source: True
mid-itf/skysimctl/4 Uncorrelated_Noise_Sources: False
mid-itf/skysimctl/4 H_Channel: False
mid-itf/skysimctl/4 V_Channel: False
mid-itf/skysimctl/4 temperature: 24.2
mid-itf/skysimctl/4 humidity: 0.0
mid-itf/siggen/1 versionId: 0.9.1
mid-itf/siggen/1 adminMode: 0
mid-itf/siggen/1 State: ON
mid-itf/siggen/1 healthState: HealthState.UNKNOWN
mid-itf/siggen/1 frequency: 800000000.0
mid-itf/siggen/1 power_cycled: False
mid-itf/siggen/1 power_dbm: -30.0
mid-itf/siggen/1 rf_output_on: True
mid-itf/siggen/1 controlMode: 0
mid-itf/siggen/1 simulationMode: 0
mid-itf/siggen/1 testMode: 0
mid-itf/siggen/1 loggingLevel: 4
mid-itf/siggen/1 command_error: False
mid-itf/siggen/1 device_error: False
mid-itf/siggen/1 execution_error: False
mid-itf/siggen/1 query_error: False
mid-itf/progattenuator/1 versionId: 0.9.1
mid-itf/progattenuator/1 model_name: MN=RCDAT-8000-30
mid-itf/progattenuator/1 adm

### 0.2.3 Create Test Equipment Plot

In [10]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()

Loading BokehJS ...

### 0.2.4 Turn offline Test Equipment devices ONLINE

In [9]:
# set any offline devices to online
test_equipment.turn_online()

set mid-itf/siggen/1 adminMode already ONLINE
set mid-itf/progattenuator/1 adminMode already ONLINE
set mid-itf/spectana/1 adminMode already ONLINE


### 0.2.5 Display Test Equipment Device States 

In [ ]:
test_equipment_state.state["devices_states"]

**Step 1:**

Set CW generator to approximately 800MHz (freq1) & -30dBm (power1), and wideband noise to approximately -132dBm/Hz at the SPF Band 1 Receiver inputs. ie. Band 1 output on and attenuation approx 10dB.  

Expected Result:  
*Test equipment configured*

In [12]:
# test equipment setup values - band 1
frequency_to_set = 800.0e6
power_level = -30.0
attenuation = 10
band = 1

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

# print current
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

# setup siggen
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", power_level)
# setup SSC
SSC.write_attribute("Correlated_Noise_Source", True)
SSC.write_attribute("Uncorrelated_Noise_Sources", False)
SSC.write_attribute("Band", band)
# setup Attenuator
prog_atten.write_attribute("channel_1", attenuation)

time.sleep(3)
# print updated values and confirm updates
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print(f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}")

assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
assert signal_generator.power_dbm == power_level, print(
    f"Power level required is {power_level} but got {signal_generator.power_dbm}"
)
assert SSC.Correlated_Noise_Source, print(
    f"Correlated noise source required is {True} but got {SSC.Correlated_Noise_Source}"
)
assert not SSC.Uncorrelated_Noise_Sources, print(
    f"Uncorrelated noise source required is {False} but got {SSC.Uncorrelated_Noise_Sources}"
)
assert SSC.Band == band, print(f"Band required is {band} but got {SSC.Band}")
assert prog_atten.channel_1 == attenuation, print(
    f"Attenuation required is {attenuation} but got {prog_atten.channel_1}"
)

Current signal generator frequency: 800000000.0
Current signal generator power level: -30.0
Current Sky Simulator Correlated Noise Source: True
Current Sky Simulator Uncorrelated Noise Source: True
Current Sky Simulator Band: 1
Current Programmable attenuator current attenuation : 10.0
Updated signal generator frequency                  : 800000000.0
Updated signal generator power level                : -30.0
Updated Sky Simulator Correlated Noise Source       : True
Updated Sky Simulator Uncorrelated Noise Source     : False
Updated Sky Simulator Band                          : 1
Updated Programmable attenuator current attenuation : 10.0


In [13]:
# print current only
signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

# print current
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

Current signal generator frequency: 800000000.0
Current signal generator power level: -30.0
Current Sky Simulator Correlated Noise Source: True
Current Sky Simulator Uncorrelated Noise Source: False
Current Sky Simulator Band: 1
Current Programmable attenuator current attenuation : 10.0


**Step 4:**

~10 seconds after recording has started, use Test Control Script to adjust CW generator signal level by -3dB. 

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

In [14]:
time.sleep(
    10
)  # cell to be run in this notebook immediately after starting scn in rf-chain-and-channelisation_test_equipment notebook
power_level_minus_3_db = power_level - 3.0
signal_generator = test_equipment.signal_generator
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute("power_dbm", power_level_minus_3_db)
time.sleep(3)
print(f"Updated signal generator power level: {signal_generator.power_dbm}")

Current signal generator power level: -30.0
Updated signal generator power level: -33.0


**Step 5:**

After a further ~10 seconds, use Test Control Script to adjust CW generator signal level by -3dB.  

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

In [15]:
time.sleep(
    10
)  # cell to be run in this notebook immediately after starting scn in rf-chain-and-channelisation_test_equipment notebook
power_level_minus_6_db = power_level - 6.0
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute("power_dbm", power_level_minus_6_db)
time.sleep(3)
print(f"Updated signal generator power level: {signal_generator.power_dbm}")

Current signal generator power level: -33.0
Updated signal generator power level: -36.0


**Step 6:**

After a further ~10seconds, use Test Control Script to increase the CW generator signal frequency by 10MHz.  

Expected Result:  
*CW frequency changes by 10MHz in SDP Spectrometer Display*

In [16]:
time.sleep(10)
frequency_plus_10MHz = frequency_to_set + 10.0e6
print(f"Current signal generator frequency: {signal_generator.frequency}")
signal_generator.write_attribute("frequency", frequency_plus_10MHz)
time.sleep(3)
print(f"Updated signal generator frequency: {signal_generator.frequency}")

Current signal generator frequency: 800000000.0
Updated signal generator frequency: 810000000.0


**Step 15:**

Set CW generator to approximately 1500MHz (freq2) & -30dBm, and wideband noise to approximately -132dBm/Hz at the SPF Band 2 Receiver inputs.  ie. Band 2 output on and attenuation approx 10dB.

Expected Result:  
*Test equipment configured*

In [17]:
# test equipment setup values - band 2
frequency_to_set = 1500.0e6
power_level = -30.0
attenuation = 10
band = 2

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

# print current
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

# setup siggen
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", power_level)
# setup SSC
SSC.write_attribute("Correlated_Noise_Source", True)
SSC.write_attribute("Uncorrelated_Noise_Sources", False)
SSC.write_attribute("Band", band)
# setup Attenuator
prog_atten.write_attribute("channel_1", attenuation)

time.sleep(3)
# print updated values and confirm updates
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print(f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}")

assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
assert signal_generator.power_dbm == power_level, print(
    f"Power level required is {power_level} but got {signal_generator.power_dbm}"
)
assert SSC.Correlated_Noise_Source, print(
    f"Correlated noise source required is {True} but got {SSC.Correlated_Noise_Source}"
)
assert not SSC.Uncorrelated_Noise_Sources, print(
    f"Uncorrelated noise source required is {False} but got {SSC.Uncorrelated_Noise_Sources}"
)
assert SSC.Band == band, print(f"Band required is {band} but got {SSC.Band}")
assert prog_atten.channel_1 == attenuation, print(
    f"Attenuation required is {attenuation} but got {prog_atten.channel_1}"
)
signal_generator = test_equipment.signal_generator
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", power_level)
time.sleep(3)
print(f"Updated signal generator frequency: {signal_generator.frequency}")
print(f"Updated signal generator power level: {signal_generator.power_dbm}")
assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
prog_atten = test_equipment.programmable_attenuator
print(prog_atten.get_attribute_list())
print(
    f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}"
)  # print current attenuationprog_atten.write_attribute('channel_1', 10)

prog_atten.write_attribute("channel_1", attenuation)
time.sleep(1)
print(
    f"Updated Programmable attenuator attenuation : {prog_atten.channel_1}"
)  # print updated attenuation"frequency_to_set = 800e6

Current signal generator frequency: 810000000.0
Current signal generator power level: -36.0
Current Sky Simulator Correlated Noise Source: True
Current Sky Simulator Uncorrelated Noise Source: False
Current Sky Simulator Band: 1
Current Programmable attenuator current attenuation : 10.0
Updated signal generator frequency                  : 1500000000.0
Updated signal generator power level                : -30.0
Updated Sky Simulator Correlated Noise Source       : True
Updated Sky Simulator Uncorrelated Noise Source     : False
Updated Sky Simulator Band                          : 2
Updated Programmable attenuator current attenuation : 10.0
Current signal generator frequency: 1500000000.0
Current signal generator power level: -30.0
Updated signal generator frequency: 1500000000.0
Updated signal generator power level: -30.0
['buildState', 'versionId', 'loggingLevel', 'loggingTargets', 'healthState', 'adminMode', 'controlMode', 'simulationMode', 'testMode', 'longRunningCommandsInQueue', 